<a href="https://colab.research.google.com/github/sanikamal/LLM-Playground/blob/main/notebooks/Mixtral_Re_Mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Mixtral with QLoRA

In [ ]:
!pip install -qU flash-attn --no-build-isolation

In [ ]:
!pip install transformers accelerate bitsandbytes peft -qU

> NOTE: We will be downloading the model's whopping ~5*19GBs of weights - and then loading those (in 4-bit quantization) into our GPU memory.

In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

bits_and_bytes_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

mixtral_7B = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bits_and_bytes_config,
    attn_implementation="flash_attention_2"
)

mixtral_tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
import transformers

text = "### Instruction:\nUse the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.\n\n### Response:"
inputs = mixtral_tokenizer(text, return_tensors="pt")

outputs = mixtral_7B.generate(**inputs, max_new_tokens=150)
print(mixtral_tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### Instruction:
Use the provided input to create an instruction that could have been used to generate the response with an LLM.### Input:
There are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.

### Response:
Based on the information provided, you could create an instruction like this:

"Generate a response describing the characteristics of four common types of grass: Kentucky Bluegrass, Ryegrass, Fescues, and Bermuda grass. Mention that Kentucky Bluegrass is the most common due to its quick and easy growth and soft texture. Describe Ryegrass as shiny and bright green. Characterize Fescues as dark green and shiny. And explain that Bermuda grass is harder and can grow in drier soil."

The LLM would then generate a response similar to the original, based on this instruction

In [ ]:
mixtral_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
!pip install -qU datasets

In [ ]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("mosaicml/instruct-v3")

In [ ]:
instruct_tune_dataset = instruct_tune_dataset.filter(lambda x: x["source"] == "dolly_hhrlhf")

In [ ]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 34333
    })
    test: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 4771
    })
})

In [ ]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(5_000))

In [ ]:
instruct_tune_dataset["test"] = instruct_tune_dataset["test"].select(range(200))

In [ ]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['prompt', 'response', 'source'],
        num_rows: 200
    })
})

In [ ]:
def create_prompt(sample):
  bos_token = "<s>"
  original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  system_message = "[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  response = sample["prompt"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
  input = sample["response"]
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += system_message
  full_prompt += "\n" + input
  full_prompt += "[/INST]"
  full_prompt += response
  full_prompt += eos_token

  return full_prompt

In [ ]:
create_prompt(instruct_tune_dataset["train"][0])

'<s>[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.[/INST]What are different types of grass?</s>'

In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(mixtral_7B)
model = get_peft_model(mixtral_7B, peft_config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_instruct_generation",
  #num_train_epochs=5,
  max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 2,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

In [ ]:
!pip install -qU trl

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=mixtral_tokenizer,
  packing=True,
  formatting_func=create_prompt,
  args=args,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:548: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:267: UserWarning: You passed 

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:570: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
T

Step,Training Loss,Validation Loss
20,3.311400,4.257724
40,3.494100,3.227834
60,3.643000,3.174503
80,3.252400,2.929174
100,3.150600,2.941102


Checkpoint destination directory mistral_instruct_generation/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=100, training_loss=3.360270538330078, metrics={'train_runtime': 482.8119, 'train_samples_per_second': 0.414, 'train_steps_per_second': 0.207, 'total_flos': 1.145886637817856e+17, 'train_loss': 3.360270538330078, 'epoch': 0.04})

In [ ]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
text = "<s>[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.[/INST]"
inputs = mixtral_tokenizer(text, return_tensors="pt")

outputs = merged_model.generate(
    **inputs,
    max_new_tokens=150,
    generation_kwargs={"repetition_penalty" : 1.7}
)
print(mixtral_tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.
There are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.[/INST]

Ryegrass is a type of grass. It is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass that is a type of grass
